In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from numpy import mean

In [2]:
LOR = pd.read_csv('./data/scores_applicant_ratings.csv')
Structured = pd.read_excel('./data/PROCESSED-data-without-TOEFL.xlsx')
struct_id = Structured['id']
unmatched_id = []
for i in range(len(LOR)):
    if LOR['id'][i] not in list(struct_id):
        unmatched_id.append(i)
LOR = LOR.drop(unmatched_id)
LOR

,filename,recommender_gender,applicant_gender,specificity,relevance,recommender_role,positivity,country,comment,rater,...,impolite_confidence,gender,race,country_of_birth,permanent_city,permanent_country,permanent_region,citizenship,undergrad1_country,undergrad1_region
7,0003806923_820037097_2022Spring_MSDS_MSDS_LOR1...,Male,Male,2,0,Academic,1,USA,NaN,David,...,0.015138,M,NaN,China,Hefei,China,Anhui,China,United States,CA
8,0003806923_820037097_2022Spring_MSDS_MSDS_LOR2...,Male,Male,1,1,Work,2,USA,NaN,David,...,0.005897,M,NaN,China,Hefei,China,Anhui,China,United States,CA
29,0006306461_560940399_2022Fall_MSDS_MSDS_LOR1_r...,Female,Male,0,0,Academic,1,USA,NaN,Ellise,...,0.010805,M,NaN,China,Panzhihua,China,Sichuan,China,United States,IN
30,0006306461_560940399_2022Fall_MSDS_MSDS_LOR2_r...,Female,Male,2,0,Academic,2,USA,NaN,Ellise,...,0.011614,M,NaN,China,Panzhihua,China,Sichuan,China,United States,IN
31,0006306461_560940399_2022Fall_MSDS_MSDS_LOR3_r...,Female,Male,2,1,Academic,1,USA,NaN,Ellise,...,0.009379,M,NaN,China,Panzhihua,China,Sichuan,China,United States,IN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3863,0764509257_983566647_2017Fall_MSCS_rejected_LO...,Male,Male,1,1,Work,1,USA,NaN,David,...,0.035201,M,NaN,China,Shanghai,China,Shanghai,China,Singapore,South West
3864,0892507559_448276861_2020Fall_MSCS_rejected_LO...,Male,Male,3,2,Academic,3,USA,very strong letter. highest overall average by...,Jason,...,0.022090,M,NaN,South Africa,Los Angeles,United States,CA,South Africa,South Africa,Western Cape
3865,0915005971_613373238_2021Spring_MSCS_rejected_...,Male,Male,1,0,Work,1,China,same as 0915005971_613373238_2020Fall_MSCS_rej...,Jason,...,0.028045,M,NaN,China,Baoding,China,Hebei,China,China,NaN
3866,0932000917_657247223_2021Spring_MSDS_MSDS_LOR5...,NaN,Female,2,0,Academic,1,USA,duplicate (same letter) of 0932000917_65724722...,Jason,...,0.008734,F,NaN,China,Zibo,China,Shandong,China,United States,OH


In [3]:
newdf = LOR[['id','length','sentiment_score', 'sadness_score', 'joy_score', 'fear_score', 'disgust_score','anger_score', 'polite_confidence', 'satisfied_confidence', 'sympathetic_confidence', 'excited_confidence', 'frustrated_confidence', 'sad_confidence', 'impolite_confidence','specificity','relevance','positivity']]
newdf

,id,length,sentiment_score,sadness_score,joy_score,fear_score,disgust_score,anger_score,polite_confidence,satisfied_confidence,sympathetic_confidence,excited_confidence,frustrated_confidence,sad_confidence,impolite_confidence,specificity,relevance,positivity
7,3806923,4531,0.810439,0.200927,0.459272,0.138154,0.043310,0.057421,0.239322,0.082862,0.029400,0.101268,0.168738,0.167017,0.015138,2,0,1
8,3806923,1981,0.749503,0.241779,0.485762,0.090809,0.034552,0.050262,0.497860,0.189657,0.170842,0.144381,0.058575,0.121240,0.005897,1,1,2
29,6306461,1264,0.733822,0.224648,0.494712,0.068431,0.050563,0.057711,0.303087,0.173891,0.061248,0.133787,0.130946,0.107361,0.010805,0,0,1
30,6306461,3031,0.884354,0.232546,0.550863,0.095990,0.034785,0.039976,0.332926,0.308889,0.210209,0.258195,0.068662,0.062324,0.011614,2,0,2
31,6306461,2810,0.858540,0.295267,0.461612,0.090294,0.037181,0.038246,0.354530,0.154248,0.130791,0.094919,0.045003,0.117701,0.009379,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3863,1764509000000,2116,0.789688,0.243905,0.450976,0.073912,0.028320,0.088337,0.185593,0.185935,0.084045,0.047949,0.230700,0.237040,0.035201,1,1,1
3864,1892508000000,2835,0.729440,0.189408,0.548351,0.082553,0.038504,0.067012,0.287390,0.101727,0.110220,0.101403,0.062616,0.117555,0.022090,3,2,3
3865,1915006000000,1797,0.702597,0.231492,0.412928,0.085706,0.048571,0.111203,0.408289,0.279813,0.101657,0.040199,0.303133,0.096969,0.028045,1,0,1
3866,1932001000000,1518,0.834917,0.273366,0.411304,0.109522,0.043429,0.056004,0.700970,0.100420,0.110065,0.188662,0.074330,0.180159,0.008734,2,0,1


In [4]:
newdf1 = pd.DataFrame()
newdf1 = pd.merge(newdf, Structured, how = 'inner', on = 'id')
newdf1['program_admitted'] = LOR['program_admitted'].tolist()

In [5]:
program_admitted_need = ['MSDS','MSCS','rejected']
newdf1 = newdf1[newdf1['program_admitted'].isin(program_admitted_need) == True]
newdf1['program_admitted'] = newdf1['program_admitted'].str.replace('MSDS','admitted')
newdf1['program_admitted'] = newdf1['program_admitted'].str.replace('MSCS','admitted')
newdf1

,id,length,sentiment_score,sadness_score,joy_score,fear_score,disgust_score,anger_score,polite_confidence,satisfied_confidence,...,Major_Flag_Engineering,Major_Flag_Finance,Major_Flag_Humanities,Major_Flag_Mathematics,Major_Flag_OTHER,Major_Flag_Physical Sciences,Major_Flag_Pyschology,undergrad1_gpa_zscore,time_since_degree_zscore,age_at_submission_zscore
0,3806923,4531,0.810439,0.200927,0.459272,0.138154,0.043310,0.057421,0.239322,0.082862,...,0,0,0,0,0,0,0,-1.040115,-0.448132,-0.585260
1,3806923,1981,0.749503,0.241779,0.485762,0.090809,0.034552,0.050262,0.497860,0.189657,...,0,0,0,0,0,0,0,-1.040115,-0.448132,-0.585260
2,6306461,1264,0.733822,0.224648,0.494712,0.068431,0.050563,0.057711,0.303087,0.173891,...,0,0,0,0,1,0,0,-0.813912,0.427874,0.640044
3,6306461,3031,0.884354,0.232546,0.550863,0.095990,0.034785,0.039976,0.332926,0.308889,...,0,0,0,0,1,0,0,-0.813912,0.427874,0.640044
4,6306461,2810,0.858540,0.295267,0.461612,0.090294,0.037181,0.038246,0.354530,0.154248,...,0,0,0,0,1,0,0,-0.813912,0.427874,0.640044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859,1592510000000,2576,0.687957,0.253960,0.435636,0.091749,0.037210,0.089535,0.531780,0.118809,...,0,0,0,0,0,0,0,-2.372196,-0.669426,-0.344495
1862,1892508000000,2835,0.729440,0.189408,0.548351,0.082553,0.038504,0.067012,0.287390,0.101727,...,0,0,0,0,0,1,0,-1.668455,-0.224544,-0.354341
1863,1915006000000,1797,0.702597,0.231492,0.412928,0.085706,0.048571,0.111203,0.408289,0.279813,...,1,0,0,0,0,0,0,-0.851110,1.618049,1.257618
1864,1932001000000,1518,0.834917,0.273366,0.411304,0.109522,0.043429,0.056004,0.700970,0.100420,...,0,0,0,1,0,0,0,0.794638,-0.183266,-0.447424


In [6]:
#X = newdf1[['length','sentiment_score', 'sadness_score', 'joy_score', 'fear_score', 'disgust_score','anger_score', 'polite_confidence', 'satisfied_confidence', 'sympathetic_confidence', 'excited_confidence', 'frustrated_confidence', 'sad_confidence', 'impolite_confidence','specificity','relevance','positivity']]
y = newdf1['program_admitted']
groups = newdf1['id']
X = newdf1.drop(['id','program_admitted','Unnamed: 0','gre_verified_verbal','gre_verified_analytical_writing','gre_verified_quantitative'],axis=1)
X

,length,sentiment_score,sadness_score,joy_score,fear_score,disgust_score,anger_score,polite_confidence,satisfied_confidence,sympathetic_confidence,...,Major_Flag_Engineering,Major_Flag_Finance,Major_Flag_Humanities,Major_Flag_Mathematics,Major_Flag_OTHER,Major_Flag_Physical Sciences,Major_Flag_Pyschology,undergrad1_gpa_zscore,time_since_degree_zscore,age_at_submission_zscore
0,4531,0.810439,0.200927,0.459272,0.138154,0.043310,0.057421,0.239322,0.082862,0.029400,...,0,0,0,0,0,0,0,-1.040115,-0.448132,-0.585260
1,1981,0.749503,0.241779,0.485762,0.090809,0.034552,0.050262,0.497860,0.189657,0.170842,...,0,0,0,0,0,0,0,-1.040115,-0.448132,-0.585260
2,1264,0.733822,0.224648,0.494712,0.068431,0.050563,0.057711,0.303087,0.173891,0.061248,...,0,0,0,0,1,0,0,-0.813912,0.427874,0.640044
3,3031,0.884354,0.232546,0.550863,0.095990,0.034785,0.039976,0.332926,0.308889,0.210209,...,0,0,0,0,1,0,0,-0.813912,0.427874,0.640044
4,2810,0.858540,0.295267,0.461612,0.090294,0.037181,0.038246,0.354530,0.154248,0.130791,...,0,0,0,0,1,0,0,-0.813912,0.427874,0.640044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859,2576,0.687957,0.253960,0.435636,0.091749,0.037210,0.089535,0.531780,0.118809,0.049349,...,0,0,0,0,0,0,0,-2.372196,-0.669426,-0.344495
1862,2835,0.729440,0.189408,0.548351,0.082553,0.038504,0.067012,0.287390,0.101727,0.110220,...,0,0,0,0,0,1,0,-1.668455,-0.224544,-0.354341
1863,1797,0.702597,0.231492,0.412928,0.085706,0.048571,0.111203,0.408289,0.279813,0.101657,...,1,0,0,0,0,0,0,-0.851110,1.618049,1.257618
1864,1518,0.834917,0.273366,0.411304,0.109522,0.043429,0.056004,0.700970,0.100420,0.110065,...,0,0,0,1,0,0,0,0.794638,-0.183266,-0.447424


# Bernouli Naive Bayes

In [7]:
from sklearn.model_selection import StratifiedKFold
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
bbc_NB = BalancedBaggingClassifier(base_estimator = BernoulliNB(), n_estimators = 100, sampler= None, random_state = 1)
originalclass = []
predictedclass = []
originalclass_10fold = []
predictedclass_10fold = []
idclass = []
for train_index, test_index in cv.split(X,y,groups):
    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]
    groups_train = groups.iloc[train_index]
    groups_test = groups.iloc[test_index]
    bbc_NB.fit(X_train,y_train)
    y_pred_proba = bbc_NB.predict_proba(X_test)
    predictedclass.extend(y_pred_proba)
    idclass.extend(groups_test)

In [8]:
predictedclass = pd.DataFrame(predictedclass, columns = ['0','1'])
predictedclass['id'] = idclass
print(predictedclass)

             0         1            id
0     0.532542  0.467458       3806923
1     0.632009  0.367991     301000214
2     0.201464  0.798536     442000785
3     0.651481  0.348519     469004599
4     0.667609  0.332391     577501796
...        ...       ...           ...
1814  0.148424  0.851576  836505000000
1815  0.494652  0.505348  840507000000
1816  0.217324  0.782676  850002000000
1817  0.227935  0.772065  889505000000
1818  0.627299  0.372701  892508000000

[1819 rows x 3 columns]


In [9]:
predictedclass['id']

0            3806923
1          301000214
2          442000785
3          469004599
4          577501796
            ...     
1814    836505000000
1815    840507000000
1816    850002000000
1817    889505000000
1818    892508000000
Name: id, Length: 1819, dtype: int64

In [10]:
def find_majority(arr):
    arr_count = Counter(arr)
    return arr_count.most_common(1)[0][0]

id_list = predictedclass['id'].unique()
voting = []
id_voting = []
for i in id_list:
    x = predictedclass.loc[predictedclass['id'] == i]
    #print(x)  #x is a dataframe
    if len(x) % 2 == 0:
        a = x[['0','1']].max()
        voting.append(a.idxmax())
        id_voting.append(x.iloc[0]['id'])
    
    else: #if the number of letters is even, we should choose the one with the highest confidence 
        id_voting.append(x.iloc[0]['id'])
        x = x.drop('id', axis = 1)
        arr = x.idxmax(axis ='columns')
        voting.append(find_majority(arr))
voting
#change to the same label
for b in range(len(voting)):
    if voting[b] == '0':
        voting[b] = 'admitted'
    else:
        voting[b] = 'rejected'

In [11]:
idclass = pd.DataFrame(idclass, columns =['id'])
idclass = idclass['id'].unique()
idclass = pd.DataFrame(idclass, columns =['id'])

In [12]:
newdf2= newdf1.drop_duplicates(subset = ['id'])
y1 = newdf2[['program_admitted','id']]
y1

,program_admitted,id
0,admitted,3806923
2,admitted,6306461
5,admitted,7304608
9,admitted,267007152
11,admitted,290003850
...,...,...
1854,admitted,978506000000
1857,rejected,1592510000000
1862,admitted,1892508000000
1863,admitted,1915006000000


In [13]:
b = pd.merge(idclass, y1, how="inner", on='id')
label = b['program_admitted']

In [14]:
print(classification_report(label,voting))

              precision    recall  f1-score   support

    admitted       0.94      0.68      0.79       633
    rejected       0.22      0.67      0.33        84

    accuracy                           0.68       717
   macro avg       0.58      0.67      0.56       717
weighted avg       0.85      0.68      0.73       717



# Logistic Regression

In [15]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
bbc_LR = BalancedBaggingClassifier(base_estimator = LR, n_estimators = 100, sampler= None, random_state = 1)
originalclass = []
predictedclass = []
originalclass_10fold = []
predictedclass_10fold = []
idclass = []
for train_index, test_index in cv.split(X,y,groups):
    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]
    groups_train = groups.iloc[train_index]
    groups_test = groups.iloc[test_index]
    bbc_LR.fit(X_train,y_train)
    y_pred_proba = bbc_LR.predict_proba(X_test)
    predictedclass.extend(y_pred_proba)
    idclass.extend(groups_test)

/Users/chenxiaoyu/.pyenv/versions/3.9.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chenxiaoyu/.pyenv/versions/3.9.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

In [16]:
predictedclass = pd.DataFrame(predictedclass, columns = ['0','1'])
predictedclass['id'] = idclass
print(predictedclass)

             0         1            id
0     0.737147  0.262853       3806923
1     0.587808  0.412192     301000214
2     0.294056  0.705944     442000785
3     0.892126  0.107874     469004599
4     0.894235  0.105765     577501796
...        ...       ...           ...
1814  0.423694  0.576306  836505000000
1815  0.472209  0.527791  840507000000
1816  0.716553  0.283447  850002000000
1817  0.006265  0.993735  889505000000
1818  0.201042  0.798958  892508000000

[1819 rows x 3 columns]


In [17]:
def find_majority(arr):
    arr_count = Counter(arr)
    return arr_count.most_common(1)[0][0]

id_list = predictedclass['id'].unique()
voting = []
id_voting = []
for i in id_list:
    x = predictedclass.loc[predictedclass['id'] == i]
    #print(x)  #x is a dataframe
    if len(x) % 2 == 0:
        a = x[['0','1']].max()
        voting.append(a.idxmax())
        id_voting.append(x.iloc[0]['id'])
    
    else: #if the number of letters is even, we should choose the one with higher confidence 
        id_voting.append(x.iloc[0]['id'])
        x = x.drop('id', axis = 1)
        arr = x.idxmax(axis ='columns')
        voting.append(find_majority(arr))
voting
#change to the same label
for b in range(len(voting)):
    if voting[b] == '0':
        voting[b] = 'admitted'
    else:
        voting[b] = 'rejected'

In [18]:
for i in range(len(id_voting)):
    id_voting[i] = int(id_voting[i])
id_voting[0]

3806923

In [19]:
idclass = pd.DataFrame(idclass, columns =['id'])
idclass = idclass['id'].unique()
idclass = pd.DataFrame(idclass, columns =['id'])

In [20]:
newdf2= newdf1.drop_duplicates(subset = ['id'])
y1 = newdf2[['program_admitted','id']]
y1

,program_admitted,id
0,admitted,3806923
2,admitted,6306461
5,admitted,7304608
9,admitted,267007152
11,admitted,290003850
...,...,...
1854,admitted,978506000000
1857,rejected,1592510000000
1862,admitted,1892508000000
1863,admitted,1915006000000


In [21]:
b = pd.merge(idclass, y1, how="inner", on='id')
label = b['program_admitted']

In [22]:
print(classification_report(label,voting))

              precision    recall  f1-score   support

    admitted       0.97      0.80      0.87       633
    rejected       0.34      0.79      0.48        84

    accuracy                           0.80       717
   macro avg       0.65      0.79      0.68       717
weighted avg       0.89      0.80      0.83       717



# Decision Tree

In [24]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
bbc_DT = BalancedBaggingClassifier(base_estimator = DT, n_estimators = 100, sampler = None, random_state = 1)
originalclass = []
predictedclass = []
originalclass_10fold = []
predictedclass_10fold = []
idclass = []
for train_index, test_index in cv.split(X,y,groups):
    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]
    groups_train = groups.iloc[train_index]
    groups_test = groups.iloc[test_index]
    bbc_DT.fit(X_train,y_train)
    y_pred_proba = bbc_DT.predict_proba(X_test)
    predictedclass.extend(y_pred_proba)
    idclass.extend(groups_test)

In [25]:
predictedclass = pd.DataFrame(predictedclass, columns = ['0','1'])
predictedclass['id'] = idclass
print(predictedclass)

         0     1            id
0     0.50  0.50       3806923
1     0.30  0.70     301000214
2     0.70  0.30     442000785
3     0.84  0.16     469004599
4     0.83  0.17     577501796
...    ...   ...           ...
1814  0.39  0.61  836505000000
1815  0.78  0.22  840507000000
1816  0.93  0.07  850002000000
1817  0.02  0.98  889505000000
1818  0.62  0.38  892508000000

[1819 rows x 3 columns]


In [26]:
def find_majority(arr):
    arr_count = Counter(arr)
    return arr_count.most_common(1)[0][0]

id_list = predictedclass['id'].unique()
voting = []
id_voting = []
for i in id_list:
    x = predictedclass.loc[predictedclass['id'] == i]
    #print(x)  #x is a dataframe
    if len(x) % 2 == 0:
        a = x[['0','1']].max()
        voting.append(a.idxmax())
        id_voting.append(x.iloc[0]['id'])
    
    else: #if the number of letters is even, we should choose the one with higher confidence 
        id_voting.append(x.iloc[0]['id'])
        x = x.drop('id', axis = 1)
        arr = x.idxmax(axis ='columns')
        voting.append(find_majority(arr))
voting
#change to the same label
for b in range(len(voting)):
    if voting[b] == '0':
        voting[b] = 'admitted'
    else:
        voting[b] = 'rejected'

In [27]:
for i in range(len(id_voting)):
    id_voting[i] = int(id_voting[i])
id_voting[0]

3806923

In [28]:
idclass = pd.DataFrame(idclass, columns =['id'])
idclass = idclass['id'].unique()
idclass = pd.DataFrame(idclass, columns =['id'])

In [29]:
newdf2= newdf1.drop_duplicates(subset = ['id'])
y1 = newdf2[['program_admitted','id']]
y1

,program_admitted,id
0,admitted,3806923
2,admitted,6306461
5,admitted,7304608
9,admitted,267007152
11,admitted,290003850
...,...,...
1854,admitted,978506000000
1857,rejected,1592510000000
1862,admitted,1892508000000
1863,admitted,1915006000000


In [30]:
b = pd.merge(idclass, y1, how="inner", on='id')
label = b['program_admitted']

In [31]:
print(classification_report(label,voting))

              precision    recall  f1-score   support

    admitted       0.98      0.87      0.92       633
    rejected       0.46      0.86      0.60        84

    accuracy                           0.87       717
   macro avg       0.72      0.86      0.76       717
weighted avg       0.92      0.87      0.88       717



# Random Forest

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
bbc_rf = BalancedBaggingClassifier(base_estimator = RandomForestClassifier(), n_estimators = 100, sampler = None, random_state = 1)
originalclass = []
predictedclass = []
originalclass_10fold = []
predictedclass_10fold = []
idclass = []
for train_index, test_index in cv.split(X,y,groups):
    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]
    groups_train = groups.iloc[train_index]
    groups_test = groups.iloc[test_index]
    bbc_rf.fit(X_train,y_train)
    y_pred_proba = bbc_rf.predict_proba(X_test)
    predictedclass.extend(y_pred_proba)
    idclass.extend(groups_test)

In [33]:
predictedclass = pd.DataFrame(predictedclass, columns = ['0','1'])
predictedclass['id'] = idclass
print(predictedclass)

           0       1            id
0     0.5635  0.4365       3806923
1     0.4378  0.5622     301000214
2     0.5677  0.4323     442000785
3     0.6911  0.3089     469004599
4     0.6958  0.3042     577501796
...      ...     ...           ...
1814  0.4955  0.5045  836505000000
1815  0.6088  0.3912  840507000000
1816  0.6933  0.3067  850002000000
1817  0.1510  0.8490  889505000000
1818  0.5361  0.4639  892508000000

[1819 rows x 3 columns]


In [34]:
#predictedclass['id']

In [35]:
def find_majority(arr):
    arr_count = Counter(arr)
    return arr_count.most_common(1)[0][0]

id_list = predictedclass['id'].unique()
voting = []
id_voting = []
for i in id_list:
    x = predictedclass.loc[predictedclass['id'] == i]
    #print(x)  #x is a dataframe
    if len(x) % 2 == 0:
        a = x[['0','1']].max()
        voting.append(a.idxmax())
        id_voting.append(x.iloc[0]['id'])
    
    else: #if the number of letters is even, we should choose the one with higher confidence 
        id_voting.append(x.iloc[0]['id'])
        x = x.drop('id', axis = 1)
        arr = x.idxmax(axis ='columns')
        voting.append(find_majority(arr))
voting
#change to the same label
for b in range(len(voting)):
    if voting[b] == '0':
        voting[b] = 'admitted'
    else:
        voting[b] = 'rejected'

In [36]:
for i in range(len(id_voting)):
    id_voting[i] = int(id_voting[i])
id_voting[0]

3806923

In [37]:
idclass = pd.DataFrame(idclass, columns =['id'])
idclass = idclass['id'].unique()
idclass = pd.DataFrame(idclass, columns =['id'])

In [38]:
newdf2= newdf1.drop_duplicates(subset = ['id'])
y1 = newdf2[['program_admitted','id']]
y1

,program_admitted,id
0,admitted,3806923
2,admitted,6306461
5,admitted,7304608
9,admitted,267007152
11,admitted,290003850
...,...,...
1854,admitted,978506000000
1857,rejected,1592510000000
1862,admitted,1892508000000
1863,admitted,1915006000000


In [39]:
b = pd.merge(idclass, y1, how="inner", on='id')
label = b['program_admitted']

In [40]:
print(classification_report(label,voting))

              precision    recall  f1-score   support

    admitted       0.97      0.90      0.93       633
    rejected       0.51      0.82      0.63        84

    accuracy                           0.89       717
   macro avg       0.74      0.86      0.78       717
weighted avg       0.92      0.89      0.90       717



Adjust threshold

In [41]:
predictedclass

,0,1,id
0,0.5635,0.4365,3806923
1,0.4378,0.5622,301000214
2,0.5677,0.4323,442000785
3,0.6911,0.3089,469004599
4,0.6958,0.3042,577501796
...,...,...,...
1814,0.4955,0.5045,836505000000
1815,0.6088,0.3912,840507000000
1816,0.6933,0.3067,850002000000
1817,0.1510,0.8490,889505000000


In [42]:
# change threshold to 0.58
for i in range(len(predictedclass['0'])):
    if predictedclass['0'].iloc[i] < 0.57 and predictedclass['0'].iloc[i] >0.5:
        predictedclass['0'].iloc[i] -= (predictedclass['0'].iloc[i]-0.5)*3
predictedclass       

/var/folders/rn/hqg5g1z50v75q_j28q166yc40000gn/T/ipykernel_26532/470782140.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictedclass['0'].iloc[i] -= (predictedclass['0'].iloc[i]-0.5)*3


,0,1,id
0,0.3730,0.4365,3806923
1,0.4378,0.5622,301000214
2,0.3646,0.4323,442000785
3,0.6911,0.3089,469004599
4,0.6958,0.3042,577501796
...,...,...,...
1814,0.4955,0.5045,836505000000
1815,0.6088,0.3912,840507000000
1816,0.6933,0.3067,850002000000
1817,0.1510,0.8490,889505000000


In [43]:
def find_majority(arr):
    arr_count = Counter(arr)
    return arr_count.most_common(1)[0][0]

id_list = predictedclass['id'].unique()
voting = []
id_voting = []
for i in id_list:
    x = predictedclass.loc[predictedclass['id'] == i]
    #print(x)  #x is a dataframe
    if len(x) % 2 == 0:
        a = x[['0','1']].max()
        voting.append(a.idxmax())
        id_voting.append(x.iloc[0]['id'])
    
    else: #if the number of letters is even, we should choose the one with higher confidence 
        id_voting.append(x.iloc[0]['id'])
        x = x.drop('id', axis = 1)
        arr = x.idxmax(axis ='columns')
        voting.append(find_majority(arr))
voting
#change to the same label
for b in range(len(voting)):
    if voting[b] == '0':
        voting[b] = 'admitted'
    else:
        voting[b] = 'rejected'

In [44]:
for i in range(len(id_voting)):
    id_voting[i] = int(id_voting[i])

In [45]:
idclass = pd.DataFrame(idclass, columns =['id'])
idclass = idclass['id'].unique()
idclass = pd.DataFrame(idclass, columns =['id'])

In [46]:
newdf2= newdf1.drop_duplicates(subset = ['id'])
y1 = newdf2[['program_admitted','id']]
y1

,program_admitted,id
0,admitted,3806923
2,admitted,6306461
5,admitted,7304608
9,admitted,267007152
11,admitted,290003850
...,...,...
1854,admitted,978506000000
1857,rejected,1592510000000
1862,admitted,1892508000000
1863,admitted,1915006000000


In [47]:
b = pd.merge(idclass, y1, how="inner", on='id')
label = b['program_admitted']

In [48]:
print(classification_report(label,voting))

              precision    recall  f1-score   support

    admitted       0.99      0.83      0.90       633
    rejected       0.41      0.92      0.57        84

    accuracy                           0.84       717
   macro avg       0.70      0.87      0.74       717
weighted avg       0.92      0.84      0.86       717



# XGBoost

In [50]:
from xgboost import XGBClassifier
XGB = XGBClassifier()
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
bbc_XGB = BalancedBaggingClassifier(base_estimator = XGB, n_estimators = 100, sampler = None, random_state = 1)
originalclass = []
predictedclass = []
originalclass_10fold = []
predictedclass_10fold = []
idclass = []
for train_index, test_index in cv.split(X,y,groups):
    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]
    groups_train = groups.iloc[train_index]
    groups_test = groups.iloc[test_index]
    bbc_XGB.fit(X_train,y_train)
    y_pred_proba = bbc_XGB.predict_proba(X_test)
    predictedclass.extend(y_pred_proba)
    idclass.extend(groups_test)

In [51]:
predictedclass = pd.DataFrame(predictedclass, columns = ['0','1'])
predictedclass['id'] = idclass
print(predictedclass)

             0         1            id
0     0.584301  0.415699       3806923
1     0.301464  0.698536     301000214
2     0.611027  0.388973     442000785
3     0.920667  0.079333     469004599
4     0.892072  0.107928     577501796
...        ...       ...           ...
1814  0.279300  0.720700  836505000000
1815  0.879147  0.120853  840507000000
1816  0.961365  0.038635  850002000000
1817  0.007713  0.992287  889505000000
1818  0.446121  0.553879  892508000000

[1819 rows x 3 columns]


In [52]:
def find_majority(arr):
    arr_count = Counter(arr)
    return arr_count.most_common(1)[0][0]

id_list = predictedclass['id'].unique()
voting = []
id_voting = []
for i in id_list:
    x = predictedclass.loc[predictedclass['id'] == i]
    #print(x)  #x is a dataframe
    if len(x) % 2 == 0:
        a = x[['0','1']].max()
        voting.append(a.idxmax())
        id_voting.append(x.iloc[0]['id'])
    
    else: #if the number of letters is even, we should choose the one with higher confidence 
        id_voting.append(x.iloc[0]['id'])
        x = x.drop('id', axis = 1)
        arr = x.idxmax(axis ='columns')
        voting.append(find_majority(arr))
voting
#change to the same label
for b in range(len(voting)):
    if voting[b] == '0':
        voting[b] = 'admitted'
    else:
        voting[b] = 'rejected'

In [53]:
for i in range(len(id_voting)):
    id_voting[i] = int(id_voting[i])
id_voting[0]

3806923

In [54]:
idclass = pd.DataFrame(idclass, columns =['id'])
idclass = idclass['id'].unique()
idclass = pd.DataFrame(idclass, columns =['id'])

In [55]:
newdf2= newdf1.drop_duplicates(subset = ['id'])
y1 = newdf2[['program_admitted','id']]
y1

,program_admitted,id
0,admitted,3806923
2,admitted,6306461
5,admitted,7304608
9,admitted,267007152
11,admitted,290003850
...,...,...
1854,admitted,978506000000
1857,rejected,1592510000000
1862,admitted,1892508000000
1863,admitted,1915006000000


In [56]:
b = pd.merge(idclass, y1, how="inner", on='id')
label = b['program_admitted']

In [57]:
print(classification_report(label,voting))

              precision    recall  f1-score   support

    admitted       0.98      0.87      0.92       633
    rejected       0.47      0.85      0.60        84

    accuracy                           0.87       717
   macro avg       0.72      0.86      0.76       717
weighted avg       0.92      0.87      0.89       717

